In [ ]:
import numpy as np
import pandas as pd

#import scipy
import matplotlib.pyplot as plt
#import os
#from scipy import io

import seaborn as sns

#from scipy import stats
#from scipy.stats import zscore
#from sklearn.preprocessing import StandardScaler
#from sklearn import preprocessing
#from sklearn.model_selection import train_test_split

#from sklearn.cluster import KMeans
#from sklearn import preprocessing

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Read Data
df_cdhdr = pd.read_excel('../Data1/CDHDR.XLSX')

In [ ]:
df_cdhdr.head(4)

In [ ]:
df_cdpos = pd.read_excel('../Data1/CDPOS.XLSX')

In [ ]:
df_cdpos.tail(4)

In [ ]:
print(df_cdhdr.shape)
print(df_cdpos.shape)

In [ ]:
df_ch = pd.merge(df_cdpos, df_cdhdr, how='left', on='CHANGENR')

In [ ]:
#sanity check (first number should be the higher one from before, second should be addition of the right numbers)
df_ch.shape

In [ ]:
df_ch.head(5)

In [ ]:
import missingno as msno
%matplotlib inline
msno.matrix(df_ch)

# Data Cleansing

## initial cleaning

In [ ]:
df = df_ch.copy()
print(df.shape)

In [ ]:

# Spalten die nur NAN enthalten entfernen
df = df.dropna(axis=1, how='all')


# Spalten die ausschließlich XY Werte enthalten (0 / X / 1 / etc.) dropen
df = df.loc[:, (df.astype(str) != 0).any(axis=0)]
df = df.loc[:, (df.astype(str) != '0,0').any(axis=0)]
df = df.loc[:, (df.astype(str) != '0,00').any(axis=0)]
df = df.loc[:, (df.astype(str) != '0,000').any(axis=0)]
df = df.loc[:, (df.astype(str) != 'X').any(axis=0)]
df = df.loc[:, (df.astype(str) != '1').any(axis=0)]



#spaletn entfernen die immer die selben Werte enthalten
keep_columns = [col for col in df.columns if len(df[col].unique()) > 1]
df = df[keep_columns].copy()


#print(len(df_io.index))
print(df.shape)

In [ ]:
import missingno as msno
%matplotlib inline
msno.matrix(df)

In [ ]:
df.head(4)

In [ ]:
# have lost somhow pretty much of the data due to the sparcity and empty cols

In [ ]:
#lets add the time and date column
#as Timestamp
df['UDATE'] = df_ch['UDATE'].copy()
#df['TMSTMP'] = pd.to_datetime(df['CPUTM'].astype(str)+' '+df['CPUDT'].astype(str))
df['TMSTMP'] = pd.to_datetime(df['UDATE'].astype(str) + df['UTIME'].astype(str) , format='%Y-%m-%d%H:%M:%S')

In [ ]:
df.head(4)

In [ ]:
#sanity check
df.shape

## data types correction

In [ ]:
df.describe(include='all').T

In [ ]:
df.sample(10)

In [ ]:
#2 Del
#ZEILE LINE_ID CPUTM_MKPF
#df = df.drop(['BUZEI'], axis=1)

In [ ]:
cont_Cols = ['CHANGENR']

#to many we create this list later by computation
cat_Cols = [ ]

num_Cols = []

dat_Cols = ['UDATE', 'UTIME', 'TMSTMP']



In [ ]:
list1 = df.columns.to_list()
print(list1)

#we already have these cols in num or cont or dat so lets make a list and remove them from all other than we have cat cols
unw = num_Cols + dat_Cols + cont_Cols

#cat cols are all cols - the cols we already defined as num / dat / cat
cat_Cols = [x for x in list1 if x not in unw]

In [ ]:
print(cat_Cols)

In [ ]:
#sanity check of cols
print(len(cont_Cols)+len(cat_Cols)+len(num_Cols)+len(dat_Cols))
print(df.shape[1]) 

In [ ]:
# Change dtype to Category for relevant features
df_cor = df.copy()
df_cor.head()

# we have to change NaN top 0 to avoid the infer dtype "object" for some cols
#df_cor['ZUONR'] = df_cor['ZUONR'].fillna(0)


# change Dtype to Cat for categorical columns
cats_cat = df_cor[cat_Cols]
        
for (columnName, columnData) in cats_cat.items():
    df_cor[columnName] = df_cor[columnName].astype(str)
    if isinstance(df_cor[columnName][0], str):
        s = pd.Series(df_cor[columnName], dtype="category")
        df_cor[columnName] = df_cor[columnName].astype('category')



In [ ]:
#check 1
df_cor.dtypes

In [ ]:
#check 3
df_cor.shape

In [ ]:
import missingno as msno
%matplotlib inline
msno.matrix(df_cor)

In [ ]:
df_cor.to_feather('../Data/D3_CDHDR-CDPOS_raw.ftr')